In [ ]:
teed_filename = 'data/teed_full.csv'

In [ ]:
import pandas as pd

df = pd.read_csv(teed_filename, sep=',')

df.columns = ['filename', 'condition', 'rat', 'rat_group', 'stim_type', 'max_amplitude', 'power']
df = df.explode('power')
df['power'] = df['power'].astype('float')

In [ ]:
import matplotlib.pyplot as plt
import ratdata.plot as rdplot
import seaborn as sns

order_labels = ['on-off', 'proportional']
plt.figure(figsize=(10, 10))
sns.boxplot(x='stim_type', y='power', data=df, order=order_labels, palette=rdplot.stim_type_palette, boxprops=dict(alpha=rdplot.boxplot_alpha))
sns.swarmplot(x='stim_type', y='power', data=df, order=order_labels, s=4, palette=rdplot.stim_type_palette)
plt.ylim([0, 110])
plt.axhline(100, color='red')
plt.xlabel('Stimulation type')
plt.ylabel('TEED in 1s compared to open-loop [%]')
plt.title('Average energy delivered in 1s of stimulation compared to continuous stimulation')
plt.savefig('plots/teed.svg', bbox_inches='tight', facecolor='white')